In [98]:
import os
import pandas as pd
import numpy as np
import skimage
from skimage import transform
import nibabel
import os
from shutil import copyfile


Centres=['KCL','MAN', 'NIJ']

Demogs='/mnt/storage/home/robert/Convnet_Classifier/euaims_ventricle_deviation_data.csv'
Demographics=pd.read_csv(Demogs)

base_dest = '/mnt/storage/home/robert/Convnet_Classifier/Convnet_Classifier/data/Ventricles_Regression/'

with open (base_dest+'/test_subs.txt', 'r') as f:
    test_ids = f.read().splitlines()
with open (base_dest+'/train_subs.txt', 'r') as f:
    train_ids = f.read().splitlines()
   

In [46]:
#Copy files to Convnet location

no_lists = []

for centre in Centres:
    filepath=('/mnt/storage/home/robert/AIMS/EUAIMS_STRUCTURAL/T1_Registered/V01/'+centre)
    files = os.listdir(filepath)
    
    c_files = list(filter(lambda k: '_Warped.nii.gz' in k, files))

    no_lists_centre = []
    for file in c_files:
        if file[:12] in test_ids:
            copyfile(os.path.join(filepath,file), os.path.join(base_dest+'/Origdata/Tests',file))
            print(file, 'tests')
        elif file[:12] in train_ids:
            copyfile(os.path.join(filepath,file), os.path.join(base_dest+'/Origdata/Trains',file))
            print(file, 'trains')
        else:
            print(file, 'no listing!')
            no_lists_centre.append(file)
            
    no_lists.append(no_lists_centre)

908722914011_T1_brain_MNI_Warped.nii.gz tests
378543460381_T1_brain_MNI_Warped.nii.gz trains
871752256167_T1_brain_MNI_Warped.nii.gz trains
611237764697_T1_brain_MNI_Warped.nii.gz tests
336598049412_T1_brain_MNI_Warped.nii.gz tests
634529090328_T1_brain_MNI_Warped.nii.gz trains
952847408985_T1_brain_MNI_Warped.nii.gz no listing!
852911509359_T1_brain_MNI_Warped.nii.gz trains
851870964066_T1_brain_MNI_Warped.nii.gz tests
942412768004_T1_brain_MNI_Warped.nii.gz trains
468991529680_T1_brain_MNI_Warped.nii.gz trains
509985289516_T1_brain_MNI_Warped.nii.gz tests
926056905071_T1_brain_MNI_Warped.nii.gz no listing!
570954148543_T1_brain_MNI_Warped.nii.gz trains
530329509553_T1_brain_MNI_Warped.nii.gz tests
468580210800_T1_brain_MNI_Warped.nii.gz trains
261249677383_T1_brain_MNI_Warped.nii.gz no listing!
198955934445_T1_brain_MNI_Warped.nii.gz trains
104324981539_T1_brain_MNI_Warped.nii.gz trains
920590066881_T1_brain_MNI_Warped.nii.gz trains
511856618487_T1_brain_MNI_Warped.nii.gz trains
4203

In [119]:
#Converting to npy and adding label
subjects=('/mnt/storage/home/robert/Convnet_Classifier/EUAIMS_MNIWARP_T01_Only_SubjectID_Order.txt')
with open (subjects, 'r') as f:
    subs=f.read().splitlines()
subs=[int(x) for x in subs]
Labels = Demographics[Demographics.subjects.isin(subs)]
cols={'subjects','dev'}
Labels=Labels[cols]

trains_data_dir=base_dest+'/Origdata/Trains'
tests_data_dir=base_dest+'/Origdata/Tests'

#labeling and object formation

for file in os.listdir(trains_data_dir):
    netdata=[]                                              #will be used for numpy object
    try:
        img = nibabel.load(os.path.join(trains_data_dir, file))  # loading the image
        img = img.get_fdata()                           # accessing image array
        img = skimage.transform.resize(img, (106, 106, 120))#resizing the image to dimensions(106,106,120)
        id = file.partition('_')
        labelar = pd.Series(Labels['dev'][Labels['subjects']==int(id[0])]).values
        netdata.append([img, labelar])                          #one hot encoding and saving numpy object
        np.save(os.path.join(base_dest+'/Trains', id[0] ), netdata)
    except:
        print(file, 'error')
        continue
        
for file in os.listdir(tests_data_dir):
    netdata=[]                                              #will be used for numpy object
    try:
        img = nibabel.load(os.path.join(tests_data_dir, file))  # loading the image
        img = img.get_fdata()                           # accessing image array
        img = skimage.transform.resize(img, (106, 106, 120))#resizing the image to dimensions(106,106,120)
        id = file.partition('_')
        labelar = pd.Series(Labels['dev'][Labels['subjects']==int(id[0])]).values
        netdata.append([img, labelar])                          #one hot encoding and saving numpy object
        np.save(os.path.join(base_dest+'/Tests', id[0] ), netdata)
    except:
        print(file, 'error')
        continue

In [120]:
#normalisation
totalnum=[]         #total number of pixels in the image
mean=[]             #mean of the pixels in the image
nummax=[]           #maximum value of pixels in the image
for file in os.listdir(base_dest+'/Trains'):
    img = np.load(os.path.join(base_dest+'/Trains',file), allow_pickle=True)
    mean.append(np.mean(img[0][0]))
    totalnum.append((img[0][0].shape[0]*img[0][0].shape[1]*img[0][0].shape[2]))
    nummax.append(np.max(img[0][0]))
for file in os.listdir(base_dest+'/Tests'):
    img = np.load(os.path.join(base_dest+'/Tests', file), allow_pickle=True)
    mean.append(np.mean(img[0][0]))
    totalnum.append((img[0][0].shape[0]*img[0][0].shape[1]*img[0][0].shape[2]))
    nummax.append(np.max(img[0][0]))
nummean=np.vdot(mean,totalnum)/np.sum(totalnum)           #mean value for the full dataset
nummax=np.max(nummax)                                     #max value for the full dataset

for file in os.listdir(base_dest+'/Trains'):
    img = np.load(os.path.join(base_dest+'/Trains',file), allow_pickle=True)
    img[0][0]=(img[0][0]-nummean)/nummax                 #normalisation(x-mean/max value)
    np.save(os.path.join(trains_data_dir,file),img)
for file in os.listdir(base_dest+'/Tests'):
    img = np.load(os.path.join(base_dest+'/Tests', file), allow_pickle=True)
    img[0][0] =(img[0][0] - nummean) / nummax
    np.save(os.path.join(base_dest+'/Tests',file),img)